In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from MyEarlyStopping import MyEarlyStopping
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from keras.optimizers import adam

import timeit
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  

# load dataset 

In [3]:
train_number = 5
train_image = 20 #10:20%, 20: 40%, 30:60%
train_image_s2 = 30 #10:20%, 20: 40%, 30:60%

## Training dataset (40%)

In [4]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_data.flow_from_directory('/data/home/cou/yuli5/Audio/Comparision/Different_Inputs/Audio_MFCC/train',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 1714 images belonging to 10 classes.


In [5]:
image_names = train_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = train_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=train_number)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [6]:
All_Inputs = [next(train_generator)[0][0] for _ in range(len(train_generator))]
All_Inputs = np.array(All_Inputs)

In [7]:
select_indexs_train = ((image_no<train_image)&(ALL_participant_class!=2))|((image_no<train_image_s2)&(ALL_participant_class==2))
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train]
Train_participant_uniform = Train_participant_class*0+1/train_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10
# sum(Train_participant_class)


## Validation dataset (20%)

In [8]:
val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data.flow_from_directory('/data/home/cou/yuli5/Audio/Comparision/Different_Inputs/Audio_MFCC/validation',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 543 images belonging to 10 classes.


In [9]:
image_names = val_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
participant_class = np.array(list(map(int, participant_class)))
command_class = val_generator.classes
Val_participant_class = tf.keras.utils.to_categorical(participant_class-1, num_classes=train_number)
Val_participant_uniform = Val_participant_class*0+1/train_number
Val_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
Val_command_uniform = Val_command_class*0+1/10

In [10]:
Val_Inputs = [next(val_generator)[0][0] for _ in range(len(val_generator))]
Val_Inputs = np.array(Val_Inputs)

## Test dataset (20%)

In [11]:
test_data = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data.flow_from_directory('/data/home/cou/yuli5/Audio/Comparision/Different_Inputs/Audio_MFCC/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 543 images belonging to 10 classes.


In [12]:
image_names = test_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
test_unit_participant_class = np.array(list(map(int, participant_class)))
test_unit_command_class = test_generator.classes
Test_participant_class = tf.keras.utils.to_categorical(test_unit_participant_class-1, num_classes=train_number)
Test_participant_uniform = Test_participant_class*0+1/train_number
Test_command_class = tf.keras.utils.to_categorical(test_unit_command_class, num_classes=10)
Test_command_uniform = Test_command_class*0+1/10

In [13]:
Test_Inputs = [next(test_generator)[0][0] for _ in range(len(test_generator))]
Test_Inputs = np.array(Test_Inputs)

In [14]:
Test_Inputs_1 = Test_Inputs[np.where(test_unit_participant_class == 1)]
Test_command_class_1 = Test_command_class[np.where(test_unit_participant_class == 1)]
Test_Inputs_2 = Test_Inputs[np.where(test_unit_participant_class == 2)]
Test_command_class_2 = Test_command_class[np.where(test_unit_participant_class == 2)]
Test_Inputs_3 = Test_Inputs[np.where(test_unit_participant_class == 3)]
Test_command_class_3 = Test_command_class[np.where(test_unit_participant_class == 3)]
Test_Inputs_4 = Test_Inputs[np.where(test_unit_participant_class == 4)]
Test_command_class_4 = Test_command_class[np.where(test_unit_participant_class == 4)]
Test_Inputs_5 = Test_Inputs[np.where(test_unit_participant_class == 5)]
Test_command_class_5 = Test_command_class[np.where(test_unit_participant_class == 5)]

# pd file store performance

In [15]:
# pd file store performance
Perfomance = pd.DataFrame(columns = ['Inputs','Time','Partcp_Acc_p15','Command_Acc_p15','Partcp_Acc_p1','Command_Acc_p1',
                                      'Partcp_Acc_p2','Command_Acc_p2','Partcp_Acc_p3','Command_Acc_p3',
                                      'Partcp_Acc_p4','Command_Acc_p4','Partcp_Acc_p5','Command_Acc_p5'])

# Perfomance = pd.read_csv('Performance_0906_Inputs.csv')
# Perfomance
                                                   

# Create model 

In [16]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


## Group model 

In [17]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(resnet_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(train_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    resnet_model = tf.keras.Model(resnet_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])


    # resnet_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + resnet_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + resnet_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs, 
                           {resnet_model.layers[-2].name:Train_participant_class, 
                            resnet_model.layers[-1].name:Train_command_class,
                            resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                            resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                            validation_data=(Val_Inputs,
                                             {resnet_model.layers[-2].name:Val_participant_class,
                                              resnet_model.layers[-1].name:Val_command_class,
                                              resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                              resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = test_unit_participant_class
    acc_p15_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)

    predictions = resnet_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_1)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([1]*len(predicted_classes))
    acc_p1_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_2)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([2]*len(predicted_classes))
    acc_p2_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_3)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([3]*len(predicted_classes))
    acc_p3_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_4)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([4]*len(predicted_classes))
    acc_p4_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_5)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([5]*len(predicted_classes))
    acc_p5_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   


    Perfomance = Perfomance.append({'Inputs': "MFCC",'Size':'20&s2_30','Time':run_time,
                                    'Partcp_Acc_p15':acc_p15_s,'Command_Acc_p15':acc_p15_c,'Partcp_Acc_p1':acc_p1_s,
                                    'Command_Acc_p1':acc_p1_c,'Partcp_Acc_p2':acc_p2_s,'Command_Acc_p2':acc_p2_c,
                                    'Partcp_Acc_p3':acc_p3_s,'Command_Acc_p3':acc_p3_c,'Partcp_Acc_p4':acc_p4_s,
                                    'Command_Acc_p4':acc_p4_c,'Partcp_Acc_p5':acc_p5_s,'Command_Acc_p5':acc_p5_c}, ignore_index=True)


    if high_acc < acc_p15_c + acc_p15_s:
        resnet_model.save('Initial_group_model_MFCC_0906.h5')
        high_acc = acc_p15_c + acc_p15_s
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('mix_20&s2_30_history1_Initial_group_model_MFCC_0906.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('mix_20&s2_30_history2_Initial_group_model_MFCC_0906.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
18/18 [==============================] - 5s 259ms/step - loss: 3.6997 - participant_output_loss: 1.4318 - command_output_loss: 2.2633 - command_output_1_loss: 5.6156e-04 - participant_output_1_loss: 0.0041 - participant_output_accuracy: 0.3891 - command_output_accuracy: 0.2191 - command_output_1_accuracy: 0.1291 - participant_output_1_accuracy: 0.2518 - val_loss: 3.6002 - val_participant_output_loss: 1.3693 - val_command_output_loss: 2.2306 - val_command_output_1_loss: 8.7145e-05 - val_participant_output_1_loss: 2.1656e-04 - val_participant_output_accuracy: 0.4088 - val_command_output_accuracy: 0.2799 - val_command_output_1_accuracy: 0.9521 - val_participant_output_1_accuracy: 0.2063
Epoch 2/10
18/18 [==============================] - 3s 183ms/step - loss: 3.2219 - participant_output_loss: 1.1260 - command_output_loss: 2.0957 - command_output_1_loss: 9.8831e-05 - participant_output_1_loss: 1.0422e-04 - participant_output_accuracy: 0.5636 - command_output_accuracy: 0.4600 - c

Epoch 3/10
18/18 [==============================] - 3s 178ms/step - loss: 0.7888 - participant_output_loss: 0.1066 - command_output_loss: 0.6820 - command_output_1_loss: 1.0201e-04 - participant_output_1_loss: 4.6787e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9927 - command_output_1_accuracy: 0.0736 - participant_output_1_accuracy: 0.1455 - val_loss: 2.3481 - val_participant_output_loss: 0.6314 - val_command_output_loss: 1.7165 - val_command_output_1_loss: 8.3702e-05 - val_participant_output_1_loss: 5.6507e-05 - val_participant_output_accuracy: 0.8785 - val_command_output_accuracy: 0.5470 - val_command_output_1_accuracy: 0.0534 - val_participant_output_1_accuracy: 0.2652
Epoch 4/10
18/18 [==============================] - 3s 152ms/step - loss: 0.6521 - participant_output_loss: 0.0880 - command_output_loss: 0.5640 - command_output_1_loss: 8.6996e-05 - participant_output_1_loss: 4.6666e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9945

Epoch 5/10
18/18 [==============================] - 3s 181ms/step - loss: 2.4640 - participant_output_loss: 0.6010 - command_output_loss: 1.8629 - command_output_1_loss: 6.1935e-05 - participant_output_1_loss: 3.1718e-05 - participant_output_accuracy: 0.8936 - command_output_accuracy: 0.8273 - command_output_1_accuracy: 9.0909e-04 - participant_output_1_accuracy: 0.1964 - val_loss: 3.0211 - val_participant_output_loss: 0.9170 - val_command_output_loss: 2.1041 - val_command_output_1_loss: 4.4115e-05 - val_participant_output_1_loss: 3.6222e-05 - val_participant_output_accuracy: 0.7587 - val_command_output_accuracy: 0.4751 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.0810
Epoch 6/10
18/18 [==============================] - 3s 188ms/step - loss: 2.2329 - participant_output_loss: 0.4575 - command_output_loss: 1.7754 - command_output_1_loss: 5.9832e-05 - participant_output_1_loss: 2.9693e-05 - participant_output_accuracy: 0.9500 - command_output_accuracy

Epoch 7/10
18/18 [==============================] - 3s 186ms/step - loss: 0.6181 - participant_output_loss: 0.0696 - command_output_loss: 0.5484 - command_output_1_loss: 5.2746e-05 - participant_output_1_loss: 3.8757e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9973 - command_output_1_accuracy: 0.1136 - participant_output_1_accuracy: 0.1818 - val_loss: 2.3866 - val_participant_output_loss: 0.5729 - val_command_output_loss: 1.8137 - val_command_output_1_loss: 3.7329e-05 - val_participant_output_1_loss: 6.2343e-05 - val_participant_output_accuracy: 0.8858 - val_command_output_accuracy: 0.5470 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1805
Epoch 8/10
18/18 [==============================] - 3s 153ms/step - loss: 0.5043 - participant_output_loss: 0.0607 - command_output_loss: 0.4435 - command_output_1_loss: 5.4130e-05 - participant_output_1_loss: 4.1014e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.

Epoch 9/10
18/18 [==============================] - 3s 145ms/step - loss: 1.7490 - participant_output_loss: 0.2628 - command_output_loss: 1.4861 - command_output_1_loss: 7.0354e-05 - participant_output_1_loss: 2.4814e-05 - participant_output_accuracy: 0.9845 - command_output_accuracy: 0.9809 - command_output_1_accuracy: 0.2191 - participant_output_1_accuracy: 0.2145 - val_loss: 2.6239 - val_participant_output_loss: 0.7010 - val_command_output_loss: 1.9229 - val_command_output_1_loss: 3.8088e-05 - val_participant_output_1_loss: 2.8345e-05 - val_participant_output_accuracy: 0.8379 - val_command_output_accuracy: 0.5341 - val_command_output_1_accuracy: 0.0221 - val_participant_output_1_accuracy: 0.1971
Epoch 10/10
18/18 [==============================] - 3s 179ms/step - loss: 1.5972 - participant_output_loss: 0.2119 - command_output_loss: 1.3852 - command_output_1_loss: 8.0071e-05 - participant_output_1_loss: 2.5660e-05 - participant_output_accuracy: 0.9891 - command_output_accuracy: 0.980

...

Epoch 5/10
18/18 [==============================] - 3s 173ms/step - loss: 2.3892 - participant_output_loss: 0.5918 - command_output_loss: 1.7973 - command_output_1_loss: 8.4259e-05 - participant_output_1_loss: 5.0758e-05 - participant_output_accuracy: 0.9109 - command_output_accuracy: 0.8136 - command_output_1_accuracy: 0.0364 - participant_output_1_accuracy: 0.1791 - val_loss: 2.9775 - val_participant_output_loss: 0.9042 - val_command_output_loss: 2.0732 - val_command_output_1_loss: 4.8298e-05 - val_participant_output_1_loss: 5.2377e-05 - val_participant_output_accuracy: 0.7864 - val_command_output_accuracy: 0.5433 - val_command_output_1_accuracy: 0.0074 - val_participant_output_1_accuracy: 0.1400
Epoch 6/10
18/18 [==============================] - 3s 155ms/step - loss: 2.1658 - participant_output_loss: 0.4781 - command_output_loss: 1.6875 - command_output_1_loss: 1.2918e-04 - participant_output_1_loss: 5.2527e-05 - participant_output_accuracy: 0.9464 - command_output_accuracy: 0.9345

Epoch 7/10
18/18 [==============================] - 2s 115ms/step - loss: 0.5488 - participant_output_loss: 0.0553 - command_output_loss: 0.4935 - command_output_1_loss: 3.4950e-05 - participant_output_1_loss: 3.3174e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9982 - command_output_1_accuracy: 0.0118 - participant_output_1_accuracy: 0.2000 - val_loss: 2.4216 - val_participant_output_loss: 0.6283 - val_command_output_loss: 1.7932 - val_command_output_1_loss: 4.3392e-05 - val_participant_output_1_loss: 4.5834e-05 - val_participant_output_accuracy: 0.8803 - val_command_output_accuracy: 0.5525 - val_command_output_1_accuracy: 0.0166 - val_participant_output_1_accuracy: 0.1602
Epoch 8/10
18/18 [==============================] - 3s 158ms/step - loss: 0.4629 - participant_output_loss: 0.0495 - command_output_loss: 0.4133 - command_output_1_loss: 6.1187e-05 - participant_output_1_loss: 3.0652e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9991

Epoch 9/10
18/18 [==============================] - 3s 158ms/step - loss: 1.6646 - participant_output_loss: 0.4274 - command_output_loss: 1.2371 - command_output_1_loss: 4.5530e-05 - participant_output_1_loss: 2.4039e-05 - participant_output_accuracy: 0.9827 - command_output_accuracy: 0.9909 - command_output_1_accuracy: 0.3064 - participant_output_1_accuracy: 0.1918 - val_loss: 2.6248 - val_participant_output_loss: 0.7832 - val_command_output_loss: 1.8415 - val_command_output_1_loss: 2.8499e-05 - val_participant_output_1_loss: 2.8601e-05 - val_participant_output_accuracy: 0.8545 - val_command_output_accuracy: 0.5488 - val_command_output_1_accuracy: 0.1897 - val_participant_output_1_accuracy: 0.2155
Epoch 10/10
18/18 [==============================] - 2s 138ms/step - loss: 1.4954 - participant_output_loss: 0.3800 - command_output_loss: 1.1154 - command_output_1_loss: 4.8886e-05 - participant_output_1_loss: 2.3884e-05 - participant_output_accuracy: 0.9836 - command_output_accuracy: 0.991

Epoch 4/10
18/18 [==============================] - 3s 173ms/step - loss: 2.6865 - participant_output_loss: 0.7099 - command_output_loss: 1.9766 - command_output_1_loss: 5.2943e-05 - participant_output_1_loss: 2.5280e-05 - participant_output_accuracy: 0.8682 - command_output_accuracy: 0.6973 - command_output_1_accuracy: 0.0327 - participant_output_1_accuracy: 0.1482 - val_loss: 3.0504 - val_participant_output_loss: 0.9128 - val_command_output_loss: 2.1375 - val_command_output_1_loss: 4.2426e-05 - val_participant_output_1_loss: 2.4651e-05 - val_participant_output_accuracy: 0.7919 - val_command_output_accuracy: 0.4549 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1823
Epoch 5/10
18/18 [==============================] - 3s 166ms/step - loss: 2.4780 - participant_output_loss: 0.5799 - command_output_loss: 1.8980 - command_output_1_loss: 7.7342e-05 - participant_output_1_loss: 2.3790e-05 - participant_output_accuracy: 0.9309 - command_output_accuracy: 0.

18/18 [==============================] - 2s 118ms/step - loss: 0.9391 - participant_output_loss: 0.1216 - command_output_loss: 0.8173 - command_output_1_loss: 1.0799e-04 - participant_output_1_loss: 3.6215e-05 - participant_output_accuracy: 0.9991 - command_output_accuracy: 0.9991 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.1155 - val_loss: 2.3514 - val_participant_output_loss: 0.5810 - val_command_output_loss: 1.7703 - val_command_output_1_loss: 3.5205e-05 - val_participant_output_1_loss: 4.5685e-05 - val_participant_output_accuracy: 0.8821 - val_command_output_accuracy: 0.5359 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.2818
Epoch 6/10
18/18 [==============================] - 3s 152ms/step - loss: 0.7956 - participant_output_loss: 0.1053 - command_output_loss: 0.6902 - command_output_1_loss: 6.2203e-05 - participant_output_1_loss: 3.6164e-05 - participant_output_accuracy: 0.9991 - command_output_accuracy: 0.9982 - 

In [18]:
resnet_model = tf.keras.models.load_model('Initial_group_model_MFCC_0906.h5')
predictions = resnet_model.predict(Test_Inputs)[0]
predicted_classes = np.argmax(predictions, axis=1)+1
acc_s = round(sum(x == y for x, y in zip(test_unit_participant_class, predicted_classes)) / len(test_unit_participant_class),4)

predictions = resnet_model.predict(Test_Inputs)[1]
predicted_classes = np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(test_unit_command_class, predicted_classes)) / len(test_unit_command_class),4)
overall_acc = acc_c + acc_s
print(acc_s,acc_c)

0.9503 0.9595


In [19]:
Perfomance

,Inputs,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Size
0,MFCC,71.151540,0.9521,0.9466,0.973,0.9640,0.9907,0.8426,0.9478,0.9565,0.8991,0.9908,0.95,0.98,20&s2_30
1,MFCC,70.088612,0.9595,0.9484,0.982,0.9459,0.9907,0.8519,0.9565,0.9739,0.9174,0.9908,0.95,0.98,20&s2_30
2,MFCC,68.793480,0.9466,0.9466,0.973,0.9640,0.9722,0.8426,0.9652,0.9565,0.8624,0.9908,0.96,0.98,20&s2_30
3,MFCC,67.171350,0.9503,0.9448,0.964,0.9459,1.0000,0.8241,0.9565,0.9826,0.9083,0.9908,0.92,0.98,20&s2_30
4,MFCC,63.843436,0.9521,0.9558,0.955,0.9730,0.9907,0.8704,0.9652,0.9652,0.9083,0.9908,0.94,0.98,20&s2_30
5,MFCC,62.341773,0.9503,0.9558,0.982,0.9640,0.9907,0.8889,0.9391,0.9565,0.8716,0.9908,0.97,0.98,20&s2_30
6,MFCC,63.504544,0.9595,0.9448,0.982,0.9640,0.9722,0.8333,0.9565,0.9565,0.9174,0.9908,0.97,0.98,20&s2_30
7,MFCC,61.709245,0.9466,0.9540,0.982,0.9550,0.9722,0.8889,0.9391,0.9565,0.8716,0.9908,0.97,0.98,20&s2_30
8,MFCC,51.474761,0.9448,0.9521,0.982,0.9369,0.9722,0.8796,0.9478,0.9652,0.8624,0.9908,0.96,0.99,20&s2_30
9,MFCC,61.188405,0.9503,0.9595,0.982,0.9820,0.9722,0.8519,0.9478,0.9913,0.8899,0.9908,0.96,0.98,20&s2_30


In [20]:
Perfomance.to_csv('Performance_0906_Inputs.csv',index=False)

In [21]:
best_index

9